<a href="https://colab.research.google.com/github/ichekhovskikh/word-movers-distance/blob/master/word_movers_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pymorphy2[fast]

import gensim
import os
import collections
import smart_open
import random
import json
import urllib.request
import pymorphy2
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from pymystem3 import Mystem
from gensim.models import Word2Vec
from gensim.similarities import WmdSimilarity

     |████████████████████████████████| 51kB 916kB/s 
     |████████████████████████████████| 7.1MB 3.1MB/s 
     |████████████████████████████████| 378kB 25.0MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=863913 sha256=50df62d60a7f62cafd424cbefe5256466d19cfb53498af394e9c2b149941bd8e
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#Начинаем
Для начала нам понадобится комплект документов для обучения нашей модели doc2vec. Теоретически, документ может быть чем угодно: коротким твитом из 140 символов, отдельным абзацем, новостной статьей или книгой. В NLP комплект документов часто называют корпусом.

Будем тренировать нашу модель на собственном корпусе. Этот корпус содержит 70 текстов.

И мы проверим нашу модель на глаз, используя тестовый корпус, который содержит 7 документов.

Dataset состоит из трех строк: id (идентификатор строки), text (текст статьи), tag (идентификатор самой статьи, вектор которого будем обучать)

In [0]:
#@title Введите путь к файлам исходной базы статей:
train_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/data.json' #@param {type: "string"}
test_path = 'https://raw.githubusercontent.com/ichekhovskikh/recommendation-system/master/test.json' #@param {type: "string"}

## Опредлим функцию для чтения и предварительной обработки текста
Ниже мы определяем функцию для открытия  train/test файла, предварительно обрабатываем каждый текст датасета, используя простой инструмент предварительной обработки gensim (то есть, разбиваем текст на отдельные слова, удалите знаки препинания, установите строчные буквы и т. д.), лемматизацию, удаление стоп слов и возвращаем список слов. Для обучения модели нам нужно будет связать тег с каждым документом учебного корпуса. В нашем случае тег - это идентификатор статьи.

Лемматизация каждого слова статьи:

In [0]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(words):
    for word in words:
        yield morph.parse(word)[0].normal_form

Удаление стоп слов:

In [0]:
russian_stopwords = stopwords.words("russian")

def remove_stopwords(words):
    return [word for word in words if word not in russian_stopwords]

Предобработка текста статьи:

In [0]:
def advanced_preprocess(text):
    normalized_text = gensim.utils.simple_preprocess(text)
    normalized_text = list(lemmatize(normalized_text))
    normalized_text = remove_stopwords(normalized_text)
    return normalized_text

Отрытие файла с корпусом статей:

In [0]:
def read_corpus(corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return [advanced_preprocess(article['text']) for article in corpus]

Получение исходного текста статьи по индексу:

In [0]:
def get_article_text_by_index(index, corpus_path):
    with urllib.request.urlopen(corpus_path) as corpus_url:
        corpus = json.loads(corpus_url.read().decode())
        return corpus[index]['text']

Загружаем корпуса:

In [0]:
train_corpus = read_corpus(train_path)
test_corpus = read_corpus(test_path)

Давайте посмотрим на учебный корпус:

In [0]:
print(train_corpus[:2])

[['общественный', 'жизнь', 'экономика', 'рынок', 'каждый', 'готовить', 'блин', 'сковорода', 'антипригарный', 'покра', 'тие', 'знать', 'важный', 'температура', 'приготовление', 'который', 'зависеть', 'цвета', 'форма', 'изделие', 'домаш', 'условие', 'некоторый', 'колебание', 'цвет', 'толщина', 'форма', 'вполне', 'приемлемый', 'промышленный', 'произа', 'водство', 'блин', 'сложный', 'процесс', 'неоднородность', 'готовый', 'продукт', 'становиться', 'больший', 'пра', 'пятствие', 'успешный', 'реали', 'зации', 'торговый', 'сеть', 'новое', 'технический', 'решение', 'стабилизация', 'качество', 'промышленный', 'ный', 'производство', 'блин', 'разрабо', 'таль', 'специалист', 'шведский', 'фирма', 'formcook', 'промышленный', 'печь', 'термообработка', 'самый', 'разно', 'образный', 'продукт', 'мясо', 'птица', 'рыба', 'овощ', 'пять', 'год', 'поставлять', 'россия', 'компания', 'агро', 'надёжный', 'высокопроиза', 'водительный', 'оборудование', 'успеть', 'завоевать', 'авторитет', 'российский', 'специалист'

Корпус тестирования выглядит так:

In [0]:
print(test_corpus[:2])

[['язык', 'ассемблер', 'машинный', 'ориентировать', 'язык', 'низкий', 'уровень', 'команда', 'прямо', 'соответствовать', 'отдельный', 'команда', 'машина', 'также', 'предоставлять', 'дополнительный', 'возможность', 'облегчение', 'такой', 'макрокоманда', 'выражение', 'средство', 'обеспечение', 'модульность', 'программа', 'рассматриваться', 'автокод', 'расширить', 'конструкция', 'язык', 'высокий', 'уровень', 'являться', 'существенно', 'платформо', 'зависимый', 'язык', 'ассемблер', 'различный', 'аппаратный', 'платформа', 'несовместимый', 'хотя', 'мочь', 'целое', 'подобный', 'русский', 'язык', 'именоваться', 'просто', 'ассемблер', 'типичный', 'выражение', 'тип', 'писать', 'программа', 'ассемблер', 'строго', 'говорить', 'неверный', 'ассемблер', 'именоваться', 'утилит', 'трансляция', 'программа', 'язык', 'ассемблер', 'объектный', 'код', 'компьютер'], ['перитонит', 'развиваться', 'вследствие', 'бактериальный', 'инфицирование', 'брюшной', 'полость', 'подавлять', 'большинство', 'пациент', 'перито

# Обучение модели
## Создание объекта WmdSimilarity
Теперь мы создадим модель Woc2Vec с векторным размером 300 слов и перебираем учебный корпус 100 раз. Мы устанавливаем минимальную длину слова равной двум, чтобы отбрасывать слова с очень малым количеством вхождений.

Однако это очень маленький набор данных (70 документов) с короткими текстами (несколько сотен слов). 

Словарь содержит в себе все уникальные слова, извлеченных из учебного корпуса.
Параметры обучения:
- sentences: список текстов
- size: размерность вектора
- window: максимальное расстояние между текущим и прогнозируемым словом в предложении
- min_count: игнорирует все слова с общей частотой ниже этой
- workers: количество потоков обучения.
- sg: алгоритм обучения (1 - skip-gram, 0 - CBOW)
- alpha: коэфициент обучения
- min_alpha: минимальное значение коэфициента обучения
- iter: количество эпох.

In [0]:
#@title Укажите параметры обучения модели:
vector_size = 80 #@param
window =  5 #@param
epochs =  100 #@param
min_count = 2 #@param
alpha = 0.001 #@param
learning_method = "skip-gram" #@param ["skip-gram", "CBOW"] {type:"raw"}

sg = 1
if (learning_method == "CBOW"):
    sg = 0

In [0]:
%time model = Word2Vec(
    train_corpus, 
    size=vector_size, 
    window=window, 
    min_count=min_count, 
    alpha=alpha, 
    iter=epochs, 
    sg=sg,
    workers=4)

CPU times: user 7min 28s, sys: 2.48 s, total: 7min 30s
Wall time: 3min 59s


Теперь будем использовать обученный Word2Vec для нахождения близости тектов на основе Word Mover's Distance.

In [0]:
wmd_similarity = WmdSimilarity(train_corpus, model, num_best=5)

# Оценочная модель
Чтобы оценить нашу новую модель, мы сначала выведем новые векторы для каждого документа тренировочного корпуса, сравним выведенные векторы с тренировочным корпусом.

Проверка выведенного вектора по обучающему вектору является своего рода «проверкой работоспособности» в отношении того, ведет ли модель себя адекватно, хотя и не является реальным значением «точности».

Можем взглянуть на пример:

In [0]:
doc_id = random.randint(0, len(train_corpus) - 1)
doc = train_corpus[doc_id]
sims = wmd_similarity[doc]
print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text_by_index(doc_id, train_path)))
num = 0
for index, similarity in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, get_article_text_by_index(index, train_path)))

# Тестирование модели
Используя тот же подход, что и выше, мы выведем вектор для случайно выбранного тестового документа и сравним документ с нашей моделью на глаз.

In [0]:
doc_id = random.randint(0, len(test_corpus) - 1)
doc = test_corpus[doc_id]
sims = wmd_similarity[doc]
print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА ({}): «{}»\n'.format(doc_id, get_article_text_by_index(doc_id, test_path)))
num = 0
for index, similarity in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, get_article_text_by_index(index, train_path)))

Тестирование на малых данных для выявления ошибок в ходе написания кода

In [0]:
from gensim.test.utils import common_texts

text = advanced_preprocess("test interface minors")

print('common texts = ', common_texts)
print('text = ', text)

model = Word2Vec(common_texts, size=300, window=5, min_count=1, iter=10, workers=4)
wmd_similarity = WmdSimilarity(common_texts, model, num_best=10)


sims = wmd_similarity[text]
num = 0
for index, similarity in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, common_texts[index]))


common texts =  [['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]
text =  ['test', 'interface', 'minors']
1) 0.5200894838195914: «['graph', 'minors', 'survey']»

2) 0.5141109507127716: «['human', 'interface', 'computer']»

3) 0.5138739811524689: «['graph', 'minors', 'trees']»

4) 0.488107608790098: «['eps', 'user', 'interface', 'system']»

5) 0.4183060966977872: «['survey', 'user', 'computer', 'system', 'response', 'time']»

6) 0.4173230039934082: «['system', 'human', 'system', 'eps']»

7) 0.4169508292210954: «['trees']»

8) 0.4167115485236621: «['graph', 'trees']»

9) 0.4153519965455858: «['user', 'response', 'time']»



# Поиск похожих научных документов
Выполните поиск похожих научных статей

In [0]:
#@title Укажите путь к тексту статьи в формате *.txt или введите текст статьи:
article_text = '' #@param {type: "string"}
article_path = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/PY0101EN/labs/example1.txt' #@param {type: "string"}

if (article_text == ''):
    with urllib.request.urlopen(article_path) as article_url:
      article_text = article_url.read().decode()


In [0]:
normalized_text = advanced_preprocess(article_text)
sims = wmd_similarity[normalized_text]

print('ТЕКСТ ИСХДНОГО ДОКУМЕНТА «{}»\n'.format(article_text)
num = 0
for index, similarity in sims:
    num += 1
    print(u'%s) %s: «%s»\n' % (num, similarity, get_article_text_by_index(index, train_path)))